In [59]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
normal_llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [88]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
"https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
"https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
"https://lilianweng.github.io/posts/2023-06-23-agent/",
"https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
"https://lilianweng.github.io/posts/2020-10-29-odqa/",
"https://lilianweng.github.io/posts/2020-08-06-nas/",
"https://lilianweng.github.io/posts/2018-05-05-drl-implementation/"
        ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print("splits len", len(splits))
splits[0]

splits len 459


Document(page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\

In [79]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)
#HUMAN
#You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
#Question: {question} 
#Context: {context} 
#Answer:
prompt = hub.pull("rlm/rag-prompt")

In [80]:
from langchain.chains.combine_documents import create_stuff_documents_chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

normal_chain = create_stuff_documents_chain(llm=llm,prompt=prompt,output_parser=StrOutputParser())

In [89]:
# question from: https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/
response = rag_chain.invoke("What is Jailbreak Prompting?")
print(response)

Jailbreak prompting is a type of black-box attack that adversarially triggers Language Models (LLMs) to output harmful content that should have been mitigated. This attack relies on heuristic-based prompting to "jailbreak" the built-in safety of the model. Wei et al. (2023) outlined two failure modes of LLM safety to guide the design of jailbreak attacks.


In [91]:
split_message = [
    {
            "role": "system",
            "content": "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.",
    }]
# split_message.extend([
#     {
#         "role": "system",
#         "content": chunk.page_content,
#     } for chunk in splits
# ])
split_message.extend(
    [
        {
            "role": "user",
            "content": "What is Jailbreak Prompting?"
        }
    ]
)
response = normal_llm.invoke(
    input=split_message
)

parser = StrOutputParser()
response = parser.invoke(response)

print(response)

I don't know what "Jailbreak Prompting" refers to.
